**Name:** 

**ID:**

In [ ]:
# Authenticate and authorize access
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

**BigQuery DataFrames**

- bigframes.pandas provides a pandas-compatible API for analytics.

- bigframes.ml provides a scikit-learn-like API for ML.


> https://cloud.google.com/python/docs/reference/bigframes/latest



**Get data from BigQuery**

In [ ]:
import bigframes.pandas as bf

In [ ]:
PROJECT_ID = 'ds-on-gcp-411105'

In [ ]:
REGION = 'US'

In [ ]:
bf.options.bigquery.project = PROJECT_ID
bf.options.bigquery.location = REGION

In [ ]:
df = bf.read_gbq("bigquery-public-data.ml_datasets.penguins")

In [ ]:
df.head()

In [ ]:
df.shape

**Check missing values**

In [ ]:
df.isna().sum()

**Impute missing values**

In [ ]:
# Drop rows where 'culmen_length_mm' is NaN
df = df[df['culmen_length_mm'].notnull()]

In [ ]:
# Fill NaN values in 'sex' column with mode_sex
mode_sex = df['sex'].mode()[0] # Mode can return multiple values, so we take the first one

df['sex'] = df['sex'].fillna(mode_sex)

**Train/ Test splitting**

In [ ]:
from bigframes import ml

In [ ]:
import bigframes.ml.model_selection
import bigframes.ml.ensemble
import bigframes.ml.metrics

In [ ]:
# Define your features and target
X = df.drop(columns=['species'])  # Features
y = df['species']  # Target

In [ ]:
# Split data using BigFrames
X_train, X_test, y_train, y_test = ml.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)


**Create model with Train data using bigframes.ml**

In [ ]:
import time

In [ ]:
# Create classification model
start_time = time.time()
model = ml.ensemble.RandomForestClassifier(num_parallel_tree=5)  # Example classifier

# Train the model
model.fit(X_train, y_train)
training_time = time.time() - start_time

print("Training time:", training_time, "seconds")

**Predict Test data using the created model**

In [ ]:
# Predict test set
y_pred = model.predict(X_test)

In [ ]:
y_pred = y_pred['predicted_species']

**Evaluation**

**Confusion matrix**

In [ ]:
conf_matrix = ml.metrics.confusion_matrix(y_test, y_pred)

In [ ]:
conf_matrix

**Accuracy**,  **precision_score**, **recall_score**, **f1_score**

In [ ]:
#Accuracy
accuracy = ml.metrics.accuracy_score(y_test, y_pred)
print(accuracy)

In [ ]:
#Precision


In [ ]:
#Recall

In [ ]:
#F1

**Save the model in BigQuery**

In [ ]:
from google.cloud import bigquery

PROJECT_ID = 'ds-on-gcp-411105'
DATASET_ID = "DemoSupervisedML"
REGION = 'US'

client = bigquery.Client(project=PROJECT_ID)
dataset = bigquery.Dataset(PROJECT_ID + "." + DATASET_ID)
dataset.location = REGION
dataset = client.create_dataset(dataset, exists_ok=True)

print(f"Dataset {dataset.dataset_id} created.")

In [ ]:
MODEL_NAME = 'RF_predict_penguin_species'
model.to_gbq(DATASET_ID + "." + MODEL_NAME, replace=True)
print(f'https://console.developers.google.com/bigquery?p={PROJECT_ID}')